In [3]:
import cv2
import tensorflow as tf
import numpy as np
import mediapipe as mp
import pandas as pd
import datetime
from collections import defaultdict
from scipy.stats import mode

In [4]:
# Load the full model (architecture, weights, optimizer state) from the .h5 file
model = tf.keras.models.load_model("expression_model.h5")
model.load_weights("expression_model.weights.h5")

In [6]:
# Initialize Mediapipe Face Mesh and Haar Cascade for face detection
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# List of facial expression class names
class_names = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# Image preprocessing function
def preprocess_image(image):
    # Resize image to match model input size
    image = cv2.resize(image, (224, 224))
    # Add batch dimension
    image = np.expand_dims(image, axis=0)
    return image

# Initialize structure to store predictions per second
predictions_per_second = defaultdict(list)

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    # Read frame from webcam
    ret, frame = cap.read()
    # Flip frame horizontally
    frame = cv2.flip(frame, 1)
    
    # Get current timestamp
    current_time = datetime.datetime.now()
    timestamp_str = current_time.strftime("%Y-%m-%d %H:%M:%S")
    
    # Display real-time date and time on the frame with smaller font size
    cv2.putText(frame, timestamp_str, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    
    # Detect faces using Haar Cascade
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # If faces are detected
    if len(faces) > 0:
        # Select the largest face
        (x, y, w, h) = max(faces, key=lambda b: (b[2] * b[3]))
        
        # Crop the face from the frame
        face_roi = frame[y:y+h, x:x+w]
        
        # Convert face_roi to RGB for face mesh detection
        rgb_face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
        
        # Detect face mesh
        results = face_mesh.process(rgb_face_roi)
        
        # If face mesh landmarks are detected
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Draw face mesh landmarks in the top-right corner of the frame
                for landmark in face_landmarks.landmark:
                    landmark_x = int(landmark.x * 100) + frame.shape[1] - 150
                    landmark_y = int(landmark.y * 100) + 20
                    cv2.circle(frame, (landmark_x, landmark_y), 1, (0, 255, 0), -1)
                
                # Preprocess the face image
                processed_face = preprocess_image(face_roi)
                
                # Make predictions
                predictions = model.predict(processed_face)
                
                # Display class probabilities with a bar chart
                bar_width = 100
                bar_height = 15
                bar_x_offset = landmark_x - 110
                for i, prob in enumerate(predictions[0]):
                    # Display class name
                    cv2.putText(frame, class_names[i], (bar_x_offset, landmark_y + 100 + i*40 + 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
                    
                    # Calculate bar length based on probability
                    bar_length = int(prob * bar_width)
                    # Draw the bar chart
                    cv2.rectangle(frame, (bar_x_offset, landmark_y + 100 + i*40 + 25), (bar_x_offset + bar_length, landmark_y + 100 + i*40 + bar_height + 25), (255, 255, 255), -1)
                    # Display probability percentage next to the bar
                    cv2.putText(frame, f'{prob*100:.1f}%', (bar_x_offset + bar_width + 5, landmark_y + 100 + i*40 + 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
        
        # Display predicted facial expression above the face
        predicted_class_idx = np.argmax(predictions)
        predicted_class = class_names[predicted_class_idx]
        cv2.putText(frame, f'Expression: {predicted_class}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        
        # Add bounding box around the detected face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2) 

        # Store prediction results by second
        current_second = current_time.strftime("%Y-%m-%d %H:%M:%S")
        predictions_per_second[current_second].append(predicted_class)
    else:
        # If no face detected in the current second
        current_second = current_time.strftime("%Y-%m-%d %H:%M:%S")
        if current_second not in predictions_per_second:
            predictions_per_second[current_second] = ['No Face Detected']
    
    # Display the frame
    cv2.imshow('Expression Detection with Face Mesh', frame)
    
    # Exit loop if 'Enter' key is pressed
    if cv2.waitKey(1) == 13:
        break

# Release webcam and close OpenCV window
cap.release()
cv2.destroyAllWindows()

# Construct dataframe from the predictions per second
rows = []
for second, predictions in predictions_per_second.items():
    # Get the most frequent prediction for each second
    mode_prediction = mode(predictions)[0][0]
    rows.append({'time_stamp': second, 'predicted_expression': mode_prediction})

df = pd.DataFrame(rows)

# Save the dataframe to a CSV file
df.to_csv('predicted_expressions.csv', index=False)

1/1 [==============================] - 0s 41ms/step


C:\Users\Matthew\AppData\Local\Temp\ipykernel_14376\1357129501.py:117: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode_prediction = mode(predictions)[0][0]
C:\Users\Matthew\AppData\Local\Temp\ipykernel_14376\1357129501.py:117: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  mode_prediction = mode(predictions)[0][0]
